## `__subclasshook__`实现被隐式继承的类
> 采用`__subclasshook__`实现类似golang中的接口。

### 1. 鸭子类型duck typing
> 当看到一只鸟走起来像鸭子、游泳起来像鸭子、叫起来也像鸭子，那么这只鸟就可以被称为鸭子。

In [1]:
# 在python中只要实现了某个特殊方法，就会有某个行为，示例：
class Duck:
    
    def __len__(self):
        return 0
    
    def __getitem__(self, item):
        return item

In [2]:
d = Duck()

In [3]:
d

In [4]:
len(d)

0

In [5]:
d[100]

100

### 2. Golang中的接口
> Golang中并不用显性的说要实现哪个接口，只要实现了接口中有的方法，那么这个结构体就相当于实现了这接口。

#### 2-1： 先编写一个接口

```golang
stuct DuckInterface interface {
    Run()
    Call()
}
```

#### 2-2: 创建个结构体并实现接口

```golang
type XiaoHuangYa struct {
	name string
	age  uint
}

func (xhy *XiaoHuangYa) Swim() {
	fmt.Println("我是小黄鸭，我能游泳")
}

func (xhy *XiaoHuangYa) Call() {
	fmt.Printf("我是小黄鸭(%s-%d)，我能叫\n", xhy.name, xhy.age)
}

func DuckInfo(duck DuckInterface) {
	duck.Swim()
	duck.Call()
}
```

#### 2-3全部代码

```golang
package main

import "fmt"

type DuckInterface interface {
	Swim()
	Call()
}

type XiaoHuangYa struct {
	name string
	age  uint
}

func (xhy *XiaoHuangYa) Swim() {
	fmt.Println("我是小黄鸭，我能游泳")
}

func (xhy *XiaoHuangYa) Call() {
	fmt.Printf("我是小黄鸭(%s-%d)，我能叫\n", xhy.name, xhy.age)
}

func DuckInfo(duck DuckInterface) {
	duck.Swim()
	duck.Call()
}

func main() {
	fmt.Printf("===开始===\n")
	xhy := XiaoHuangYa{"test", 2}
	DuckInfo(&xhy)
	fmt.Printf("===End===\n")

}
```

#### 2-4：输出结果

```
===开始===
我是小黄鸭，我能游泳
我是小黄鸭(test-2)，我能叫
===End===

```

### 3. 使用抽象类

In [6]:
# 定义鸭子这个抽象的类
from abc import ABCMeta, abstractmethod

In [7]:
# 定义个鸭子类型
class Duck(metaclass=ABCMeta):
    
    @abstractmethod
    def swim(self):
        pass
    
    @abstractmethod
    def call(self):
        pass

In [8]:
try:
    d = Duck()
except Exception as e:
    print(e)

Can't instantiate abstract class Duck with abstract methods call, swim


In [9]:
# 继承Duck，并实现Duck类型的抽象方法
class XiaoHuangYa(Duck):
    
    def swim(self):
        print("我是小黄鸭，能游泳")
        
    def call(self):
        print("我是小黄鸭，我会叫")

In [10]:
xhy = XiaoHuangYa()

In [11]:
xhy

In [12]:
xhy.swim()
xhy.call()

我是小黄鸭，能游泳
我是小黄鸭，我会叫


In [13]:
# 判断是否是其子类
issubclass(XiaoHuangYa, Duck)

True

### 4. 使用`__subclasshook__`实现隐式类
> 无需显示的继承，通过`issubclass`也可判断其是其子类

In [14]:
# 引入abc
from abc import ABCMeta, abstractmethod

In [15]:
# 定义个检查类是否具有某个函数的方法
def _check_methods(C, *methods):
    mro = C.__mro__
    for method in methods:
        for B in mro:
            if method in B.__dict__:
                if B.__dict__[method] is None:
                    return None
                # 只要有某个父类有这个方法就break
                break
        else:
            return None
    return True

In [16]:
# 定义DuckBase的类
class DuckBase(metaclass=ABCMeta):
    
    @abstractmethod
    def swim(self):
        pass
    
    @abstractmethod
    def call(self):
        pass
    
    @classmethod
    def __subclasshook__(cls, subclass):
        if cls is DuckBase:
            # 判断类中是否有某个函数
            return _check_methods(cls, "swim", "call")
        else:
            return None

In [17]:
# 创建个类XiaoHuangYa02，隐式继承DuckBase
class XiaoHuangYa02:
    
    def swim(self):
        print("我是小黄鸭2，我会游泳")
    
    def call(self):
        print("我是小黄鸭2，我会叫")

In [18]:
# 实例化XiaoHuangYa02
xhy2 = XiaoHuangYa02()

In [19]:
xhy2

In [20]:
# 判断XiaoHuangYa02是谁的子类
issubclass(XiaoHuangYa02, DuckBase)

True

In [21]:
issubclass(XiaoHuangYa02, Duck)

False

In [22]:
# XiaoHuangYa虽然是集成了Duck抽象类，但是其实现了swim和call的方法，所以其也是DuckBase的子类
issubclass(XiaoHuangYa, DuckBase)

True

**到这里，我们通过`__subclasshook__`也实现了隐式继承**

### 5. 查看python源码

In [23]:
# 引入collections.abc中的类
from collections.abc import Sized

#### 查看源码:

- **内建常量：***

```python
NotImplemented = None # (!) real value is ''
```
---

- **Sized类：**

```python
class Sized(metaclass=ABCMeta):

    __slots__ = ()

    @abstractmethod
    def __len__(self):
        return 0

    @classmethod
    def __subclasshook__(cls, C):
        if cls is Sized:
            return _check_methods(C, "__len__")
        return NotImplemented
```

> 通过源码查看，只要实现了`__len__`这个方法，可以说就实现了Sized
---

- **_check_methods方法：**

```python
def _check_methods(C, *methods):
    mro = C.__mro__
    for method in methods:
        for B in mro:
            if method in B.__dict__:
                if B.__dict__[method] is None:
                    return NotImplemented
                break
        else:
            return NotImplemented
    return True
```



In [24]:
# 自定义一个类
class Team:
    
    def __init__(self, users):
        self.users = users
    
    def __len__(self):
        return len(self.users)

In [25]:
t = Team(users=["codelieche", "Python", "Golang"])

In [26]:
len(t)

3

In [27]:
issubclass(Team, Sized)

True